# Dự đoán các căn bệnh dựa trên dữ liệu thời tiết và khí hậu

Notebook này hướng dẫn quy trình dự đoán các căn bệnh có thể xảy ra dựa trên dữ liệu thời tiết và sức khỏe từ file `global_climate_health_impact_tracker_2015_2025.csv`.

## 1. Import thư viện cần thiết và tải dữ liệu

- Sử dụng pandas, numpy, matplotlib, seaborn, scikit-learn.
- Đọc file `global_climate_health_impact_tracker_2015_2025.csv` vào DataFrame.

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Đọc dữ liệu
file_path = 'data/global_climate_health_impact_tracker_2015_2025.csv'
df = pd.read_csv(file_path)
df.head()

## 2. Khám phá và tiền xử lý dữ liệu

- Kiểm tra giá trị thiếu, kiểu dữ liệu, thống kê cơ bản.
- Làm sạch và tiền xử lý dữ liệu (xử lý missing value, mã hóa biến phân loại...).

In [ ]:
# Thông tin tổng quan về dữ liệu
df.info()

# Thống kê mô tả các trường số
df.describe()

# Kiểm tra giá trị thiếu
df.isnull().sum()

# Hiển thị các giá trị duy nhất của các trường dạng object (chuỗi)
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].unique()[:10]}")  # Hiển thị 10 giá trị đầu tiên

In [ ]:
# Tiền xử lý dữ liệu
# Ví dụ: điền giá trị thiếu, mã hóa biến phân loại

df_clean = df.copy()

# Điền giá trị thiếu cho các trường số bằng trung bình
for col in df_clean.select_dtypes(include=[np.number]).columns:
    df_clean[col].fillna(df_clean[col].mean(), inplace=True)

# Điền giá trị thiếu cho các trường object bằng mode
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

# Mã hóa biến phân loại (nếu cần)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = le.fit_transform(df_clean[col])

df_clean.head()
df_clean.columns

## 3. Trích xuất và xây dựng đặc trưng (Feature Engineering)

- Lựa chọn hoặc tạo các đặc trưng liên quan đến thời tiết, khí hậu, vùng miền,... ảnh hưởng đến bệnh.

In [ ]:
# Giả sử cột 'disease' là nhãn cần dự đoán, các cột còn lại là đặc trưng
# (Có thể điều chỉnh lại tên cột cho phù hợp với dữ liệu thực tế)

features = [col for col in df_clean.columns if col != 'disease']
X = df_clean[features]
y = df_clean['disease']

print('Các đặc trưng sử dụng:', features)
print('Số lượng nhãn bệnh:', y.nunique())

## 4. Chia dữ liệu thành tập huấn luyện và kiểm tra

- Sử dụng train_test_split để chia dữ liệu.

In [ ]:
# Giới hạn số lượng hàng để train
n_samples = 10000
X_limited = X.iloc[:n_samples]
y_limited = y.iloc[:n_samples]

X_train, X_test, y_train, y_test = train_test_split(
    X_limited, y_limited, test_size=0.2, random_state=42)

print('Kích thước tập train:', X_train.shape)
print('Kích thước tập test:', X_test.shape)

## 5. Huấn luyện mô hình dự đoán bệnh

- Sử dụng mô hình phân loại (RandomForestClassifier, LogisticRegression, ...).

In [ ]:
# Huấn luyện mô hình Random Forest cho multi-output regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

clf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
clf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = clf.predict(X_test)

## 6. Đánh giá hiệu quả mô hình

- Sử dụng các chỉ số: accuracy, precision, recall, F1-score, confusion matrix.

In [ ]:
# Đánh giá mô hình cho từng nhãn bệnh
from sklearn.metrics import mean_squared_error, r2_score
for i, label in enumerate(label_cols):
    print(f'--- {label} ---')
    print('MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()

## 7. Dự đoán bệnh dựa trên dữ liệu thời tiết mới

- Sử dụng mô hình đã huấn luyện để dự đoán bệnh cho dữ liệu mới hoặc tập test.

In [ ]:
# Dự đoán bệnh cho tập test (hoặc dữ liệu mới)
predicted_diseases = clf.predict(X_test)

# Hiển thị kết quả dự đoán đầu tiên cho từng nhãn bệnh
for i, label in enumerate(label_cols):
    print(f'--- {label} ---')
    print('Giá trị thực tế:', y_test.iloc[:10, i].values)
    print('Giá trị dự đoán:', predicted_diseases[:10, i])
    print()

## 8. Trực quan hóa kết quả dự đoán

- Vẽ biểu đồ phân bố các loại bệnh được dự đoán, heatmap, v.v.

In [ ]:
# Trực quan hóa kết quả dự đoán cho từng nhãn bệnh
import matplotlib.pyplot as plt
for i, label in enumerate(label_cols):
    plt.figure(figsize=(8,4))
    plt.scatter(y_test.iloc[:, i], predicted_diseases[:, i], alpha=0.5)
    plt.xlabel('Giá trị thực tế')
    plt.ylabel('Giá trị dự đoán')
    plt.title(f'Dự đoán cho {label}')
    plt.plot([y_test.iloc[:, i].min(), y_test.iloc[:, i].max()], [y_test.iloc[:, i].min(), y_test.iloc[:, i].max()], 'r--')
    plt.show()

In [ ]:
# Heatmap hệ số quan trọng của các đặc trưng với từng vấn đề sức khỏe
importances = []
for i, label in enumerate(label_cols):
    # Lấy feature importances trung bình từ tất cả các estimator của RandomForest
    rf_list = clf.estimators_[i].estimators_ if hasattr(clf.estimators_[i], 'estimators_') else [clf.estimators_[i]]
    importances_rf = np.mean([rf.feature_importances_ for rf in rf_list], axis=0)
    importances.append(importances_rf)

importances = np.array(importances)

plt.figure(figsize=(12, 6))
sns.heatmap(importances, annot=True, fmt='.2f', cmap='YlOrRd', xticklabels=feature_cols, yticklabels=label_cols)
plt.title('Heatmap: Độ quan trọng của các đặc trưng với từng vấn đề sức khỏe')
plt.xlabel('Đặc trưng (Feature)')
plt.ylabel('Vấn đề sức khỏe (Label)')
plt.tight_layout()
plt.show()